In [ ]:
from gensim import corpora
from gensim.models import LdaModel
import json
from data_analysis.OpeningLDA import OpeningLDA
from classes.QueryHandler import QueryHandler

## LDA

In [ ]:
#opening_lda = OpeningLDA(num_passes=5, print_debug=True) 

#print(len(opening_lda.texts.keys()))
#opening_lda.lda_model.print_topics(num_words=15)

#opening_lda.save('lda')


In [ ]:
opening_lda = OpeningLDA.load("lda")

In [ ]:
opening_lda.__construct_topic_doc_matrix__()

In [ ]:
query_handler = QueryHandler()
query_handler.opening_lda = opening_lda

sample_query1 = { 
                 "message": "capture the center, develop light square bishop",
                 "color": "white" 
}
query_handler.handle_user_query(sample_query1, debug=True)

## GPT

In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from scraper.Paths import Paths
from scraper.Scraper import Scraper
from math import ceil, floor

In [2]:
test_input = open(f"{Paths.RAW_DESC_BASE}/A27/wikipedia.txt").read().replace("\n", "")

# Choose a model and load tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Create a summarization pipeline using the loaded model and tokenizer
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)


In [3]:
input_toks = tokenizer.tokenize(test_input)                 # Tokens in ENTIRE input string
num_toks = len(tokenizer.tokenize(test_input[:1024]))       # Number of tokens in the first 1024 CHARACTERS of input string
n = sum([len(t) for t in input_toks[:1024]])                # Sum of the lengths of the first 1024 tokens
num_chars = n - floor(n/num_toks)*2                          # ???

summary = summarizer(test_input[:num_chars], max_length=1024, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

print(f"num_toks =  Number of tokens in the first 1024 CHARACTERS of input string \n\t= {num_toks}\n") 
print(f"n = Sum of the lengths of the first 1024 tokens\n\t= {n}\n")
print(f"num_chars = number of characters passed to summarizer, needs to be close to 1024 without going over\n\tn - floor(n/num_toks)*2\n\t= {num_chars}\n")
print(f"# of tokens summarizer processes (i.e. the number of tokens in the first [num_chars] characters of the input string) \n\t= \n")
# Print the generated summary
print(summary[0]['summary_text'])

Token indices sequence length is longer than the specified maximum sequence length for this model (4869 > 1024). Running this sequence through the model will result in indexing errors
Your max_length is set to 1024, but your input_length is only 1017. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=508)


['Che', 'ss', 'Ġopening', 'English', 'ĠOpening', 'abc', 'def', 'gh', '88', '77', '66', '55', '44', '33', '22', '11', 'abc', 'def', 'gh', 'M', 'oves', '1', '.', 'c', '4', 'EC', 'OA', '10', 'âĢĵ', 'A', '39', 'Origin', 'St', 'aun', 'ton', 'Ġvs', '.', 'ĠSaint', '-', 'Am', 'ant', ',', 'Ġ18', '43', 'N', 'amed', 'Ġafter', 'Howard', 'ĠSt', 'aun', 'ton', ',', 'ĠEnglish', 'Ġplayer', 'Ġand', 'ĠWorld', 'ĠChampion', 'Ġ(', 'un', 'official', ')', 'The', 'ĠEnglish', 'ĠOpening', 'Ġis', 'Ġa', 'Ġchess', 'Ġopening', 'Ġthat', 'Ġbegins', 'Ġwith', 'Ġthe', 'Ġmove', ':', '1', '.', 'Ġc', '4', 'A', 'Ġflank', 'Ġopening', ',', 'Ġit', 'Ġis', 'Ġthe', 'Ġfourth', 'Ġmost', 'Ġpopular', '[', '1', '][', '2', ']', 'Ġand', ',', 'Ġaccording', 'Ġto', 'Ġvarious', 'Ġdatabases', ',', 'Ġone', 'Ġof', 'Ġthe', 'Ġfour', 'Ġmost', 'Ġsuccessful', 'Ġof', 'ĠWhite', "'s", 'Ġtwenty', 'Ġpossible', 'Ġfirst', 'Ġmoves', '.[', '1', '][', '3', ']', 'ĠWhite', 'Ġbegins', 'Ġthe', 'Ġfight', 'Ġfor', 'Ġthe', 'Ġcentre', 'Ġby', 'Ġst', 'aking', 'Ġa', 'Ġcl

In [ ]:
model_name:str = "facebook/bart-large-xsum"

max_input_size:int = 15000
tokenizer = AutoTokenizer.from_pretrained(model_name, max_length=max_input_size)
summarizer = pipeline("summarization", model=model_name, tokenizer=tokenizer)

In [ ]:
input_text = open(Paths.RAW_DESC_BASE + "C16/wikipedia.txt", "r", encoding="utf-8").read().replace("\n", "")
input_toks = tokenizer.tokenize(input_text, )

summary = summarizer(input_text, max_length=200, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)

# Print the generated summary
print(summary)

In [ ]:
input_toks = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(input_text)))
input_len = len(input_toks) + 10
summaries:list[str] = []

n:int = ceil(input_len / max_sum_len)                     # Number of sub summaries needed

print(f"Max len: {ma}")
print(f"Len input toks: {input_len}")
print(f"n = {n}")

for j in range(0,n):
    these_toks:list[str] = input_toks[j*max_len : (j*max_len) + max_len - 1]
    this_text:str = " ".join(these_toks)
    
    print(f"Processing text {j}/{n} (len = {len(these_toks)})")
    
    this_sum:str = summarizer(this_text, max_length=200, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)
    summaries.append(this_sum[0]['summary_text'])



In [ ]:
input_text = open(Paths.RAW_DESC_BASE + "A04/chess.com.txt", "r", encoding="utf-8").read().replace("\n", "")

input_ids = bart_tokenizer.encode(input_text, return_tensors="pt", max_length=200, truncation=True)

print(len(input_ids[0]))
print(bart_tokenizer.decode(input_ids[0], skip_special_tokens=True))

cleaned_input_text = bart_tokenizer.decode(input_ids[0], skip_special_tokens=True)

summary = bart_model.summarizer(input_text, max_length=200, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)

print(summary)